## Pokémon Damage Simulation with a Simplified Formula

This notebook simulates Pokémon battle damage using a simplified version of the official damage formula. The calculation is based on the following core stats:

- Level of the attacking Pokémon

- Move power

- Attacker's offensive stat (Attack or Special Attack)

- Defender's defensive stat (Defense or Special Defense)

- Type multipliers (STAB × effectiveness)

This version excludes elements that add variability and complexity in actual games, such as:

- The random damage factor (between 0.85 and 1.00)

- Held items, abilities, weather conditions.

- Critical effect (between x1,5 or x2)

#### Why a Simplified Damage Formula?

The goal of this simulation is to estimate generic damage values that can be applied across most situations, without the variability and complexity of real in-game mechanics.

By focusing on the main stats (level, power, attack, defense, STAB, and type effectiveness), we can approximate how strong a move is in general terms.

This allows us to compare moves or Pokémon consistently and interpretable, without needing to simulate all conditional modifiers like abilities, items, or weather.

### 1. Importing Libraries and Loading Pokémon Data
In this section, we import the necessary libraries and load the preprocessed Pokémon dataset (pokemon_data_mod.csv) along with the move information file (movimientos_pokemon_info.csv).

We also display the structure of both DataFrames to verify the loaded data.

In [3]:
###///Librerias///###
import pandas as pd

dat_base = pd.read_csv("../data/pokemon_data_mod.csv")
info_movimientos = pd.read_csv("../data/movimientos_pokemon_info.csv")

dat_base.info()
info_movimientos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1302 entries, 0 to 1301
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               1302 non-null   int64 
 1   name             1302 non-null   object
 2   base_experience  1302 non-null   int64 
 3   height           1302 non-null   int64 
 4   weight           1302 non-null   int64 
 5   abilities        1302 non-null   object
 6   primary_type     1302 non-null   object
 7   secondary_type   726 non-null    object
 8   move_1           1268 non-null   object
 9   move_2           1265 non-null   object
 10  move_3           1263 non-null   object
 11  move_4           1263 non-null   object
 12  move_5           1262 non-null   object
 13  hp               1302 non-null   int64 
 14  attack           1302 non-null   int64 
 15  defense          1302 non-null   int64 
 16  special-attack   1302 non-null   int64 
 17  special-defense  1302 non-null   

### 2. Selecting a Test Pokémon and Retrieving Its Moves
In this step, we select a single Pokémon from the dataset (pokemon_test) to simulate damage output.
We extract the Pokémon's move list and use it to gather detailed information about each move from the info_movimientos DataFrame.

Moves that are not found in the move info database are skipped with a warning, and we filter out any moves lacking a defined power value,
since they are not usable for damage calculation.

In [5]:
#Seleccionamos el pokemon del cual vamos a calcular el daño
pokemon_test = dat_base.iloc[30]

#Cogemos la lista de movimientos del pokemon y obtenemos los datos de este que estan en info_movimientos
df_moves = pd.DataFrame()
lista_moves=[]

lista_moves = [pokemon_test["move_1"], pokemon_test["move_2"], pokemon_test["move_3"], pokemon_test["move_4"], pokemon_test["move_5"]]

for mov in lista_moves:
    # Filtrar la fila en info_movimientos donde el nombre es igual a mov
    fila = info_movimientos[info_movimientos['name'] == mov]
    
    if not fila.empty:
        df_moves = pd.concat([df_moves, fila], ignore_index=True)
        
    else:
        print(f"Movimiento {mov} no encontrado en info_movimientos")
print(df_moves)

#Borramos aquellos movimientos que no tienen poder de ataque
df_moves = df_moves[df_moves["power"].notna()].reset_index(drop=True)
print(df_moves)

            name  power  accuracy  pp  priority      type damage_class
0     mega-punch   80.0      85.0  20         0    normal     physical
1        pay-day   40.0     100.0  20         0    normal     physical
2     fire-punch   75.0     100.0  15         0      fire     physical
3      ice-punch   75.0     100.0  15         0       ice     physical
4  thunder-punch   75.0     100.0  15         0  electric     physical
            name  power  accuracy  pp  priority      type damage_class
0     mega-punch   80.0      85.0  20         0    normal     physical
1        pay-day   40.0     100.0  20         0    normal     physical
2     fire-punch   75.0     100.0  15         0      fire     physical
3      ice-punch   75.0     100.0  15         0       ice     physical
4  thunder-punch   75.0     100.0  15         0  electric     physical


### 3. Type Effectiveness Table
A dictionary named efectividad is defined to represent the damage relationships between Pokémon types.

Each key corresponds to the attack move’s type, and each subkey indicates the defender’s type along with the damage multiplier (e.g., 2.0 for super effective, 0.5 for not very effective, and 0.0 for no effect).

This table is essential for calculating the type modifier in damage formulas.

In [7]:
#tabla efectividad tipos
efectividad = {
    'normal':   {'rock': 0.5, 'ghost': 0.0, 'steel': 0.5},
    'fire':     {'fire': 0.5, 'water': 0.5, 'grass': 2.0, 'ice': 2.0, 'bug': 2.0, 'rock': 0.5, 'dragon': 0.5, 'steel': 2.0},
    'water':    {'fire': 2.0, 'water': 0.5, 'grass': 0.5, 'ground': 2.0, 'rock': 2.0, 'dragon': 0.5},
    'electric': {'water': 2.0, 'electric': 0.5, 'grass': 0.5, 'ground': 0.0, 'flying': 2.0, 'dragon': 0.5},
    'grass':    {'fire': 0.5, 'water': 2.0, 'grass': 0.5, 'poison': 0.5, 'ground': 2.0, 'flying': 0.5, 'bug': 0.5, 'rock': 2.0, 'dragon': 0.5, 'steel': 0.5},
    'ice':      {'fire': 0.5, 'water': 0.5, 'grass': 2.0, 'ice': 0.5, 'ground': 2.0, 'flying': 2.0, 'dragon': 2.0, 'steel': 0.5},
    'fighting': {'normal': 2.0, 'ice': 2.0, 'poison': 0.5, 'flying': 0.5, 'psychic': 0.5, 'bug': 0.5, 'rock': 2.0, 'ghost': 0.0, 'dark': 2.0, 'steel': 2.0, 'fairy': 0.5},
    'poison':   {'grass': 2.0, 'poison': 0.5, 'ground': 0.5, 'rock': 0.5, 'ghost': 0.5, 'steel': 0.0, 'fairy': 2.0},
    'ground':   {'fire': 2.0, 'electric': 2.0, 'grass': 0.5, 'poison': 2.0, 'flying': 0.0, 'bug': 0.5, 'rock': 2.0, 'steel': 2.0},
    'flying':   {'electric': 0.5, 'grass': 2.0, 'fighting': 2.0, 'bug': 2.0, 'rock': 0.5, 'steel': 0.5},
    'psychic':  {'fighting': 2.0, 'poison': 2.0, 'psychic': 0.5, 'dark': 0.0, 'steel': 0.5},
    'bug':      {'fire': 0.5, 'grass': 2.0, 'fighting': 0.5, 'poison': 0.5, 'flying': 0.5, 'psychic': 2.0, 'ghost': 0.5, 'dark': 2.0, 'steel': 0.5, 'fairy': 0.5},
    'rock':     {'fire': 2.0, 'ice': 2.0, 'fighting': 0.5, 'ground': 0.5, 'flying': 2.0, 'bug': 2.0, 'steel': 0.5},
    'ghost':    {'normal': 0.0, 'psychic': 2.0, 'ghost': 2.0, 'dark': 0.5},
    'dragon':   {'dragon': 2.0, 'steel': 0.5, 'fairy': 0.0},
    'dark':     {'fighting': 0.5, 'psychic': 2.0, 'ghost': 2.0, 'dark': 0.5, 'fairy': 0.5},
    'steel':    {'fire': 0.5, 'water': 0.5, 'electric': 0.5, 'ice': 2.0, 'rock': 2.0, 'steel': 0.5, 'fairy': 2.0},
    'fairy':    {'fire': 0.5, 'fighting': 2.0, 'poison': 0.5, 'dragon': 2.0, 'dark': 2.0, 'steel': 0.5},
}

### 4. STAB Calculation (Same Type Attack Bonus)
This section calculates the STAB multiplier for each move of the selected Pokémon.

A move receives a 1.5× damage boost if its type matches either of the Pokémon's types (primary_type or secondary_type).

The script iterates through the selected Pokémon's moves, checks for type match, assigns the appropriate STAB value (1.5 or 1), and stores the result in a new column stab in df_moves.

In [9]:
#STAB
stab_values = []
tipo_ataque=[]

tipo_ataque = [pokemon_test["primary_type"], pokemon_test["secondary_type"]]

for _, row in df_moves.iterrows():
    mov_type = row["type"]  # Tipo del movimiento
    if mov_type in tipo_ataque:    # ¿Está en los tipos del Pokémon?
        stab = 1.5
        print(f"Mismo tipo: {mov_type}")
    else:
        stab = 1
        print(f"Diferente tipo: {mov_type}")
    stab_values.append(stab)

# Añadir la columna STAB al DataFrame
df_moves["stab"] = stab_values

print(df_moves)

Diferente tipo: normal
Diferente tipo: normal
Diferente tipo: fire
Diferente tipo: ice
Diferente tipo: electric
            name  power  accuracy  pp  priority      type damage_class  stab
0     mega-punch   80.0      85.0  20         0    normal     physical     1
1        pay-day   40.0     100.0  20         0    normal     physical     1
2     fire-punch   75.0     100.0  15         0      fire     physical     1
3      ice-punch   75.0     100.0  15         0       ice     physical     1
4  thunder-punch   75.0     100.0  15         0  electric     physical     1


### 5. Type Effectiveness Multiplier Function
This function calculates the total type effectiveness multiplier based on the move's type and the defender's type(s).

It uses a nested dictionary (efectividad) that encodes how effective each attacking type is against each defending type. For dual-type defenders, it multiplies the effectiveness values for both types. If any type is missing (None or NaN), the function treats it as neutral (1.0).

This multiplier is a crucial factor in determining final damage output.

In [11]:
def obtener_multiplicador(tipo_movimiento, tipos_defensor, efectividad):
    """
    Calcula el multiplicador total de efectividad dado un tipo de ataque
    y una lista de tipos de defensa.

    Args:
        tipo_movimiento (str): tipo del movimiento (ataque).
        tipos_defensor (list): lista de tipos del Pokémon defensor.
        efectividad (dict): diccionario de efectividad tipo vs tipo.

    Returns:
        float: multiplicador total.
    """
    if tipo_movimiento is None or tipo_movimiento != tipo_movimiento:  # NaN check
       return 1.0

    multiplicador = 1.0
    for tipo_def in tipos_defensor:
        if tipo_def is None or tipo_def != tipo_def:  # NaN check
            continue
        mult = efectividad.get(tipo_movimiento, {}).get(tipo_def, 1.0)
        multiplicador *= mult

    return multiplicador

### 5.1. Compute Type Multipliers Against All Pokémon
This loop **evaluates how each move of the selected Pokémon would perform against every other Pokémon in the dataset**.

For each move, the script:

- Extracts its **type, power, STAB multiplier, and damage class**.

- Iterates through all other Pokémon, retrieving their primary and secondary types, as well as their physical and special defenses.

- Calculates the type effectiveness multiplier using the helper function **obtener_multiplicador()**.

All this information is stored in a new DataFrame df_mult, where each row represents a specific move against a specific defending Pokémon.

In [13]:
#Bucle para obtener los multiplicadores con todos los pokemons

#pokemon seleccionado
print("El pokémon seleccionado:",pokemon_test['name'])

# Lista para guardar filas
filas = []

for _, mov in df_moves.iterrows():
    tipo_mov = mov['type']
    nombre_mov = mov['name']
    stab = mov['stab']
    poder = mov['power']
    clase_daño = mov['damage_class']
    
    for _, poke in dat_base.iterrows():
        nombre_poke = poke['name']
        tipos_defensor = [poke['primary_type'], poke['secondary_type']]
        defensa = poke['defense']
        def_esp = poke['special-defense']
        mult = obtener_multiplicador(tipo_mov, tipos_defensor, efectividad)
        
        filas.append({
            "pokemon_defensor": nombre_poke,
            "nombre_mov": nombre_mov,
            "movimiento_tipo": tipo_mov,
            "stab": stab,
            "poder": poder,
            "clase_daño": clase_daño,
            "tipos_defensor": tipos_defensor,
            "def": defensa,
            "def_esp": def_esp,
            "multiplicador": mult
        })

# Crear DataFrame con toda la info
df_mult = pd.DataFrame(filas)

print(df_mult)

El pokémon seleccionado: nidoqueen
              pokemon_defensor     nombre_mov movimiento_tipo  stab  poder  \
0                    bulbasaur     mega-punch          normal     1   80.0   
1                      ivysaur     mega-punch          normal     1   80.0   
2                     venusaur     mega-punch          normal     1   80.0   
3                   charmander     mega-punch          normal     1   80.0   
4                   charmeleon     mega-punch          normal     1   80.0   
...                        ...            ...             ...   ...    ...   
6505   ogerpon-wellspring-mask  thunder-punch        electric     1   75.0   
6506  ogerpon-hearthflame-mask  thunder-punch        electric     1   75.0   
6507  ogerpon-cornerstone-mask  thunder-punch        electric     1   75.0   
6508        terapagos-terastal  thunder-punch        electric     1   75.0   
6509         terapagos-stellar  thunder-punch        electric     1   75.0   

     clase_daño   tipos_defe

### 6. Base Damage Calculation Function
This function **implements a simplified version of the Pokémon damage formula**, where the final damage output is calculated using the attacker's level, move power, offensive stat (Attack or Special Attack), the defender's corresponding defensive stat, and the combined multiplier of STAB and type effectiveness.

*The formula does not include randomness (the 0.85–1.00 factor in the actual games), nor does it account for situational elements such as held items, abilities, or weather effects. It aims to offer a generic estimation of damage across most typical battle scenarios.

In [15]:
def calcular_dano_base(nivel, poder, ataque_atacante, defensa_defensor, multiplicador_total):
    """
    Fórmula simplificada de daño base en Pokémon.
    """
    dano = (((2 * nivel / 5 + 2) * poder * ataque_atacante / defensa_defensor) / 50 + 2) * multiplicador_total
    return dano

### 6.1. Loop to Estimate Damage Against All Opponents
In this section, we simulate how much damage the selected Pokémon would deal to every other Pokémon in the dataset using each of its valid offensive moves.
The loop retrieves the necessary attributes — move power, type, STAB, and offensive/defensive stats — and applies the simplified damage formula to compute an estimated damage value.

We differentiate between physical and special moves to choose the correct attacking and defending stats. The resulting estimated damage values are saved into a new DataFrame (df_result) for further inspection or comparison.

This allows us to approximate how effective each move would be against various opponents, providing a general-purpose insight into offensive capabilities.

In [17]:
###////////////Bucle para extraer los datos////////////###
nombre_ataque = pokemon_test["name"]
ataque = pokemon_test["attack"]
ataque_esp = pokemon_test["special-attack"]

filas = []

print(f"\n→ Calculando daño causado por {nombre_ataque}:\n")

nivel = 50

for _, poke in df_mult.iterrows():
    nombre_defensa = poke["pokemon_defensor"]
    poder = poke["poder"]
    clase_daño = poke["clase_daño"]
    defensa = poke["def"]
    defensa_esp = poke["def_esp"]
    mult = poke["multiplicador"]
    stab = poke["stab"]
    multiplicador_total = mult * stab

    if clase_daño == "physical":
        dano = calcular_dano_base(nivel, poder, ataque, defensa, multiplicador_total)
    else:
        dano = calcular_dano_base(nivel, poder, ataque_esp, defensa_esp, multiplicador_total)

    print(f" a {nombre_defensa} es de {dano:.2f} (Clase: {clase_daño})")
    filas.append({
        "pokemon_atacante": nombre_ataque,
        "pokemon_defensor": nombre_defensa,
        "nombre_mov": nombre_mov,
        "movimiento_tipo": tipo_mov,
        "stab": stab,
        "poder": poder,
        "clase_daño": clase_daño,
        "tipos_defensor": tipos_defensor,
        "def": defensa,
        "def_esp": defensa_esp,
        "multiplicador": mult,
        "multiplicador_total": multiplicador_total,
        "daño_estimado": dano
    })
# Crear DataFrame con toda la info
df_result = pd.DataFrame(filas)


→ Calculando daño causado por nidoqueen:

 a bulbasaur es de 68.09 (Clase: physical)
 a ivysaur es de 53.40 (Clase: physical)
 a venusaur es de 41.02 (Clase: physical)
 a charmander es de 77.31 (Clase: physical)
 a charmeleon es de 57.83 (Clase: physical)
 a charizard es de 43.52 (Clase: physical)
 a squirtle es de 51.82 (Clase: physical)
 a wartortle es de 42.48 (Clase: physical)
 a blastoise es de 34.38 (Clase: physical)
 a caterpie es de 94.53 (Clase: physical)
 a metapod es de 60.88 (Clase: physical)
 a butterfree es de 66.77 (Clase: physical)
 a weedle es de 109.95 (Clase: physical)
 a kakuna es de 66.77 (Clase: physical)
 a beedrill es de 82.96 (Clase: physical)
 a pidgey es de 82.96 (Clase: physical)
 a pidgeotto es de 60.88 (Clase: physical)
 a pidgeot es de 45.18 (Clase: physical)
 a rattata es de 94.53 (Clase: physical)
 a raticate es de 55.97 (Clase: physical)
 a spearow es de 109.95 (Clase: physical)
 a fearow es de 51.82 (Clase: physical)
 a ekans es de 75.60 (Clase: phys